In [ ]:
import numpy as np
import scipy
from scipy.stats import norm
from scipy.stats import t
import scipy.stats as stats
import pandas as pd
from matplotlib import pyplot as plt
from utils import plot_test_curve
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib import rcParams
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.genmod.bayes_mixed_glm as glm
import seaborn as sns

rcParams['savefig.dpi'] = 400
rcParams['figure.dpi'] = 400

# Combining all Data form all flight configs

In [ ]:
long_data_high_lift=pd.read_csv('metrics_data/high_lift_data_long.csv', index_col=0)
long_data_high_lift['AoA'] = 10
long_data_high_lift['gust_mag'] = [ 0.09 if c == 7.5 else 0.12 if c == 10 else 0.17 if c == 12.5 else 0 for c in long_data_high_lift['gust']]
long_data_low_lift=pd.read_csv('metrics_data/low_lift_data_long.csv', index_col=0)
long_data_low_lift['gust_mag'] = [ 0.14 if c == 4 else 0.23 if c == 6 else 0.32 if c == 8 else 0 for c in long_data_low_lift['gust']]
long_data_medium_lift=pd.read_csv('metrics_data/med_lift_data_long.csv', index_col=0)
long_data_medium_lift['gust_mag'] = [ 0.24 if c == 3 else 0.4 if c == 4.5 else 0.56 if c == 6 else 0 for c in long_data_medium_lift['gust']]

frames = [long_data_high_lift, long_data_low_lift, long_data_medium_lift]

total_data_df = pd.concat(frames).reset_index(drop=True)
total_data_df

In [ ]:
total_data_df['tap_cat'] = ['Ta' if c==6 else 'Tb' if c==3 else 'Tc' for c in total_data_df['taps']]
total_data_df['IDnum'] = [str(int(c)) for c in total_data_df['IDnum']]
total_data_df['gust_dir'] = total_data_df['gust_mag']*total_data_df['dir']
total_data_df['gustID'] = [str(int(c)) for c in total_data_df['gustID']]
total_data_df['contID'] = [str(int(c)) for c in total_data_df['contID']]
total_data_df['AoA'] = [str(int(c)) for c in total_data_df['AoA']]
total_data_df['speed'] = [str(int(c)) for c in total_data_df['speed']]
total_data_df['e_reduct'] = [ c*100 for c in total_data_df['e_reduct']]
total_data_df['flight_cond'] = total_data_df["AoA"]+'_'+total_data_df["speed"]

total_data_df['ind'] = [str(int(c)) for c in total_data_df.index]
total_data_df

In [ ]:
no_1t_df = total_data_df.copy()
ind_1t = no_1t_df[no_1t_df['taps'] == 1].index
no_1t_df.drop(ind_1t , inplace=True)

no_3t_df = total_data_df.copy()
ind_3t = no_3t_df[no_3t_df['taps'] == 3].index
no_3t_df.drop(ind_3t , inplace=True)

no_6t_df = total_data_df.copy()
ind_6t = no_6t_df[no_6t_df['taps'] == 6].index
no_6t_df.drop(ind_6t , inplace=True)


In [ ]:
total_data_df['red_mean'] = [total_data_df['e_reduct'][total_data_df.IDnum==c].mean() for c in total_data_df['IDnum']]
total_data_df['red_dev'] = abs(total_data_df['e_reduct'] - total_data_df['red_mean'])
total_data_df['rise_mean'] = [total_data_df['r_time'][total_data_df.IDnum==c].mean() for c in total_data_df['IDnum']]
total_data_df['red_std'] = [total_data_df['e_reduct'][total_data_df.IDnum==c].std() for c in total_data_df['IDnum']]
total_data_df['cont_gustID'] = total_data_df["contID"]+'_'+total_data_df["gustID"]
total_data_df['r_t_mean'] = [total_data_df['r_time'][total_data_df.cont_gustID==c].mean() for c in total_data_df['cont_gustID']]
total_data_df['rise_skew'] = [total_data_df['r_time'][total_data_df.IDnum==c].skew() for c in total_data_df['IDnum']]
total_data_df['red_skew'] = [total_data_df['e_reduct'][total_data_df.IDnum==c].skew() for c in total_data_df['IDnum']]

#total_data_df.to_csv('data/all_data_df.csv')

In [ ]:
print(total_data_df[1800:1800+18]['rise_skew'])
print(total_data_df[1800:1800+18]['rise_skew'].mean())

In [ ]:
#first = 64 #dropped outliers
first = 0 
for_stats_high_lift=pd.read_csv('metrics_data/high_lift_metrics.csv', index_col=0)
for_stats_low_lift=pd.read_csv('metrics_data/low_lift_metrics.csv', index_col=0)
for_stats_med_lift=pd.read_csv('metrics_data/med_lift_metrics.csv', index_col=0)
gusts_high_lift = [7.5, 10, 12.5]
dirs = [1,-1]
taps = [1,3,6]
gusts_low_lift = [4, 6, 8]
gusts_med_lift = [3,4.5,6]


fig, ax = plt.subplots(2,3, figsize=(8,6))

for i in range(len(gusts_high_lift)):
    gust_ind = i%len(gusts_high_lift)
    for ii in range(len(dirs)):
        dir_ind = ii%len(dirs)
    
    
        rel_stats_high_lift = for_stats_high_lift.loc[for_stats_high_lift.gust==gusts_high_lift[gust_ind]][for_stats_high_lift.dir==dirs[dir_ind]].reset_index()
     
        ax[0,0].errorbar(rel_stats_high_lift['taps'],rel_stats_high_lift['red_mean']*100,yerr=0,  # iqr,
                     color = rel_stats_high_lift['d_colors'][0],
                     marker = rel_stats_high_lift['d_marks'][0],
                     alpha = rel_stats_high_lift['alphas'][0]) #, capsize=3)
        
        ax[1,0].errorbar(rel_stats_high_lift['taps'],rel_stats_high_lift['rise_mean'],yerr=0,
                     color = rel_stats_high_lift['d_colors'][0],
                     marker = rel_stats_high_lift['d_marks'][0],
                     alpha = rel_stats_high_lift['alphas'][0])#, capsize=5)
        
        rel_stats_med_lift = for_stats_med_lift.loc[for_stats_med_lift.gust==gusts_med_lift[gust_ind]][for_stats_med_lift.dir==dirs[dir_ind]].reset_index()
        
        print(rel_stats_med_lift['red_mean']*100)
        
        ax[0,1].errorbar(rel_stats_med_lift['taps'],rel_stats_med_lift['red_mean']*100,yerr=0,  # iqr,
                     color = rel_stats_med_lift['d_colors'][0],
                     marker = rel_stats_med_lift['d_marks'][0],
                     alpha = rel_stats_med_lift['alphas'][0]) #, capsize=3)
        
        ax[1,1].errorbar(rel_stats_med_lift['taps'],rel_stats_med_lift['rise_mean'],yerr=0,
                     color = rel_stats_med_lift['d_colors'][0],
                     marker = rel_stats_med_lift['d_marks'][0],
                     alpha = rel_stats_med_lift['alphas'][0])#, capsize=5)
        
        rel_stats_low_lift = for_stats_low_lift.loc[for_stats_low_lift.gust==gusts_low_lift[gust_ind]][for_stats_low_lift.dir==dirs[dir_ind]].reset_index()

        ax[0,2].errorbar(rel_stats_low_lift['taps'],rel_stats_low_lift['red_mean']*100,yerr=0,  # iqr,
                     color = rel_stats_low_lift['d_colors'][0],
                     marker = rel_stats_low_lift['d_marks'][0],
                     alpha = rel_stats_low_lift['alphas'][0]) #, capsize=3)
        
        ax[1,2].errorbar(rel_stats_low_lift['taps'],rel_stats_low_lift['rise_mean'],yerr=0,
                     color = rel_stats_low_lift['d_colors'][0],
                     marker = rel_stats_low_lift['d_marks'][0],
                     alpha = rel_stats_low_lift['alphas'][0])#, capsize=5)
        
        
        
ax[0,0].plot(np.array([1,3,6]),np.array([1,1,1])*99,ls=(0, (3,3,3,3)), color='k')
ax[0,0].scatter(np.array([3,6]), np.array([99,99]),color='k',marker='o')
ax[0,0].scatter(np.array([1]), np.array([99]),color='k',marker='o',
                facecolor='w')

ax[0,1].plot(np.array([1,3,6]),np.array([1,1,1])*99,ls=(0, (3,3,3,3)), color='k')
ax[0,1].scatter(np.array([3,6]), np.array([99,99]),color='k',marker='o')
ax[0,1].scatter(np.array([1]), np.array([99]),color='k',marker='o',
                facecolor='w')

ax[0,2].plot(np.array([1,3,6]),np.array([1,1,1])*99,ls=(0, (3,3,3,3)), color='k')
ax[0,2].scatter(np.array([3,6]), np.array([99,99]),color='k',marker='o')
ax[0,2].scatter(np.array([1]), np.array([99]),color='k',marker='o',
                facecolor='w')

ax[1,0].plot(np.array([1,3,6]),np.array([1,1,1])*0.1,ls=(0, (3,3,3,3)), color='k')
ax[1,0].scatter(np.array([1,3,6]), np.array([0.1,0.1,0.1]),color='k',marker='o')
ax[1,1].plot(np.array([1,3,6]),np.array([1,1,1])*0.1,ls=(0, (3,3,3,3)), color='k')
ax[1,1].scatter(np.array([1,3,6]), np.array([0.1,0.1,0.1]),color='k',marker='o')

ax[1,2].plot(np.array([1,3,6]),np.array([1,1,1])*0.1,ls=(0, (3,3,3,3)), color='k')
ax[1,2].scatter(np.array([1,3,6]), np.array([0.1,0.1,0.1]),color='k',marker='o')


ax[0,0].spines["top"].set_visible(False)
ax[0,0].spines["right"].set_visible(False)
ax[0,0].spines["bottom"].set_visible(False)
ax[0,0].xaxis.set_visible(False)
ax[1,0].spines["top"].set_visible(False)
ax[1,0].spines["right"].set_visible(False)

ax[0,1].spines["top"].set_visible(False)
ax[0,1].spines["right"].set_visible(False)
ax[0,1].spines["left"].set_visible(False)
ax[0,1].spines["bottom"].set_visible(False)
ax[0,1].yaxis.set_visible(False)
ax[0,1].xaxis.set_visible(False)
ax[1,1].spines["top"].set_visible(False)
ax[1,1].spines["right"].set_visible(False)
ax[1,1].spines["left"].set_visible(False)

ax[1,1].yaxis.set_visible(False)

ax[0,0].set_ylabel('Mean Gust Reduction (%)', fontsize=14)
ax[1,0].set_ylabel('Mean Rise Time (s)', fontsize=14)
ax[1,0].set_xticks(taps)
ax[1,1].set_xticks(taps)


ax[0,0].set_ylim([40,101])
ax[1,0].set_ylim([0,1.25])
ax[0,0].set_xlim([0,7])
ax[1,0].set_xlim([0,7])

ax[0,0].set_xticks(taps)
ax[0,0].set_yticks([40,60,80,100])
ax[1,0].set_yticks([0.25,0.50,0.75,1,1.25])


ax[0,1].set_ylim([40,101])
ax[1,1].set_ylim([0,1.25])

             
ax[0,2].set_ylim([40,101])
ax[1,2].set_ylim([0,1.25])
ax[0,2].set_xlim([0,7])
ax[1,2].set_xlim([0,7])

ax[0,1].set_xlim([0,7])
ax[1,1].set_xlim([0,7])

ax[0,2].set_xticks(taps)


ax[0,2].spines["top"].set_visible(False)
ax[0,2].spines["right"].set_visible(False)
ax[0,2].spines["bottom"].set_visible(False)
ax[0,2].xaxis.set_visible(False)
ax[1,2].spines["top"].set_visible(False)
ax[1,2].spines["right"].set_visible(False)

ax[0,2].spines["top"].set_visible(False)
ax[0,2].spines["right"].set_visible(False)
ax[0,2].spines["left"].set_visible(False)
ax[0,2].spines["bottom"].set_visible(False)
ax[0,2].yaxis.set_visible(False)
ax[0,2].xaxis.set_visible(False)
ax[1,2].spines["top"].set_visible(False)
ax[1,2].spines["right"].set_visible(False)
ax[1,2].spines["left"].set_visible(False)
ax[1,2].yaxis.set_visible(False)


ax[1,1].set_xlabel('Number of taps', fontsize=14)
ax[1,2].set_xticks(taps)

fig.tight_layout()
fig.savefig('plots/e_reduct_mean_and_rise')
fig.show()

In [ ]:

first = 0 
for_stats_high_lift=pd.read_csv('metrics_data/high_lift_metrics.csv', index_col=0)
for_stats_low_lift=pd.read_csv('metrics_data/low_lift_metrics.csv', index_col=0)
for_stats_med_lift=pd.read_csv('metrics_data/med_lift_metrics.csv', index_col=0)
gusts_high_lift = [7.5, 10, 12.5]
dirs = [1,-1]
taps = [1,3,6]
gusts_low_lift = [4, 6, 8]
gusts_med_lift = [3,4.5,6]


fig, ax = plt.subplots(2,3, figsize=(8,6))

for i in range(len(gusts_high_lift)):
    gust_ind = i%len(gusts_high_lift)
    for ii in range(len(dirs)):
        dir_ind = ii%len(dirs)
    
    
        rel_stats_high_lift = for_stats_high_lift.loc[for_stats_high_lift.gust==gusts_high_lift[gust_ind]][for_stats_high_lift.dir==dirs[dir_ind]].reset_index()
     
        ax[0,0].errorbar(rel_stats_high_lift['taps'],rel_stats_high_lift['red_std']*100,yerr=0,  # iqr,
                     color = rel_stats_high_lift['d_colors'][0],
                     marker = rel_stats_high_lift['d_marks'][0],
                     alpha = rel_stats_high_lift['alphas'][0]) #, capsize=3)
        
        ax[1,0].errorbar(rel_stats_high_lift['taps'],rel_stats_high_lift['rise_std'],yerr=0,
                     color = rel_stats_high_lift['d_colors'][0],
                     marker = rel_stats_high_lift['d_marks'][0],
                     alpha = rel_stats_high_lift['alphas'][0])#, capsize=5)
        
        rel_stats_med_lift = for_stats_med_lift.loc[for_stats_med_lift.gust==gusts_med_lift[gust_ind]][for_stats_med_lift.dir==dirs[dir_ind]].reset_index()
        
        print(rel_stats_med_lift['red_mean']*100)
        
        ax[0,1].errorbar(rel_stats_med_lift['taps'],rel_stats_med_lift['red_std']*100,yerr=0,  # iqr,
                     color = rel_stats_med_lift['d_colors'][0],
                     marker = rel_stats_med_lift['d_marks'][0],
                     alpha = rel_stats_med_lift['alphas'][0]) #, capsize=3)
        
        ax[1,1].errorbar(rel_stats_med_lift['taps'],rel_stats_med_lift['rise_std'],yerr=0,
                     color = rel_stats_med_lift['d_colors'][0],
                     marker = rel_stats_med_lift['d_marks'][0],
                     alpha = rel_stats_med_lift['alphas'][0])#, capsize=5)
        
        rel_stats_low_lift = for_stats_low_lift.loc[for_stats_low_lift.gust==gusts_low_lift[gust_ind]][for_stats_low_lift.dir==dirs[dir_ind]].reset_index()

        ax[0,2].errorbar(rel_stats_low_lift['taps'],rel_stats_low_lift['red_std']*100,yerr=0,  # iqr,
                     color = rel_stats_low_lift['d_colors'][0],
                     marker = rel_stats_low_lift['d_marks'][0],
                     alpha = rel_stats_low_lift['alphas'][0]) #, capsize=3)
        
        ax[1,2].errorbar(rel_stats_low_lift['taps'],rel_stats_low_lift['rise_std'],yerr=0,
                     color = rel_stats_low_lift['d_colors'][0],
                     marker = rel_stats_low_lift['d_marks'][0],
                     alpha = rel_stats_low_lift['alphas'][0])#, capsize=5)




ax[0,0].spines["top"].set_visible(False)
ax[0,0].spines["right"].set_visible(False)
ax[0,0].spines["bottom"].set_visible(False)
ax[0,0].xaxis.set_visible(False)
ax[1,0].spines["top"].set_visible(False)
ax[1,0].spines["right"].set_visible(False)

ax[0,1].spines["top"].set_visible(False)
ax[0,1].spines["right"].set_visible(False)
ax[0,1].spines["left"].set_visible(False)
ax[0,1].spines["bottom"].set_visible(False)
ax[0,1].yaxis.set_visible(False)
ax[0,1].xaxis.set_visible(False)
ax[1,1].spines["top"].set_visible(False)
ax[1,1].spines["right"].set_visible(False)
ax[1,1].spines["left"].set_visible(False)

ax[1,1].yaxis.set_visible(False)

ax[0,0].set_ylabel('Gust Reduction STD (%)', fontsize=14)
ax[1,0].set_ylabel('Rise Time STD (s)', fontsize=14)
ax[1,0].set_xticks(taps)
ax[1,1].set_xticks(taps)


ax[0,0].set_ylim([0,45])
ax[1,0].set_ylim([0,1.6])
ax[0,0].set_xlim([0,7])
ax[1,0].set_xlim([0,7])

ax[0,0].set_xticks(taps)
ax[0,0].set_yticks([0,15,30,45])
ax[1,0].set_yticks([0, 0.4,0.8,1.2,1.6])


ax[0,1].set_ylim([0,45])
ax[1,1].set_ylim([0,1.6])

             
ax[0,2].set_ylim([0,45])
ax[1,2].set_ylim([0,1.6])
ax[0,2].set_xlim([0,7])
ax[1,2].set_xlim([0,7])

ax[0,1].set_xlim([0,7])
ax[1,1].set_xlim([0,7])

ax[0,2].set_xticks(taps)


ax[0,2].spines["top"].set_visible(False)
ax[0,2].spines["right"].set_visible(False)
ax[0,2].spines["bottom"].set_visible(False)
ax[0,2].xaxis.set_visible(False)
ax[1,2].spines["top"].set_visible(False)
ax[1,2].spines["right"].set_visible(False)

ax[0,2].spines["top"].set_visible(False)
ax[0,2].spines["right"].set_visible(False)
ax[0,2].spines["left"].set_visible(False)
ax[0,2].spines["bottom"].set_visible(False)
ax[0,2].yaxis.set_visible(False)
ax[0,2].xaxis.set_visible(False)
ax[1,2].spines["top"].set_visible(False)
ax[1,2].spines["right"].set_visible(False)
ax[1,2].spines["left"].set_visible(False)
ax[1,2].yaxis.set_visible(False)


ax[1,1].set_xlabel('Number of taps', fontsize=14)
ax[1,2].set_xticks(taps)

fig.tight_layout()
fig.savefig('plots/e_reduct_std_and_rise')
fig.show()

In [ ]:

d_colors=['#C0822C','grey','#0F665E']
d_marks = ['^','.','v']
alphas = [1, 0.6, 0.3]
fig, ax = plt.subplots(1,1, figsize=(6,3))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
legend_elements = [Line2D([0], [0], color=d_colors[0], lw=2,alpha=alphas[0],marker='^', label='+Strong'),
                   Line2D([0], [0], color=d_colors[0],lw=2,alpha=alphas[1], marker='^',label='+Moderate'),
                   Line2D([0], [0], color=d_colors[0], lw=2,alpha=alphas[2],marker='^', label='+Mild'),
                   Line2D([0], [0], color=d_colors[2], lw=2,alpha=alphas[2],marker='v', label='- Mild'),
                   Line2D([0], [0], color=d_colors[2],lw=2,alpha=alphas[1], marker='v',label='- Moderate'),
                   Line2D([0], [0], color=d_colors[2], lw=2,alpha=alphas[0],marker='v', label='- Strong')]

fig.legend(handles=legend_elements, frameon=False, bbox_to_anchor=(0.25, 0.25),
    loc=3, ncol=1, borderaxespad=0, title= '$\\bf{Gust Deflections}$')


legend_elements2=[Line2D([0], [0], marker='o', color='w', markerfacecolor='none',
                         markeredgecolor='k', label='significant',markersize=6),
                  Line2D([0], [0], marker='o', color='w', markerfacecolor='k',
                         markeredgecolor='k', label='nonsignificant',markersize=6)]

fig.legend(handles=legend_elements2, frameon=False, bbox_to_anchor=(0.50, 0.37),
    loc=3, ncol=1, borderaxespad=0, title= '$\\bf{Statistical Difference}$')

fig.tight_layout()
fig.savefig('plots/legend')

In [ ]:

d_colors=['#C0822C','grey','#0F665E']
d_marks = ['^','.','v']
alphas = [1, 0.6, 0.3]
fig, ax = plt.subplots(1,1, figsize=(6,3))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
legend_elements1 = [Line2D([0], [0], color=d_colors[0], lw=2,alpha=alphas[0], label='+Strong'),
                   Line2D([0], [0], color=d_colors[0],lw=2,alpha=alphas[1], label='+Moderate'),
                   Line2D([0], [0], color=d_colors[0], lw=2,alpha=alphas[2], label='+Mild'),
                   Line2D([0], [0], color='k',ls="-", lw=2,alpha=alphas[0], label='Average')]
legend_elements2 = [Line2D([0], [0], color=d_colors[2], lw=2,alpha=alphas[0], label='- Strong'),
                   Line2D([0], [0], color=d_colors[2],lw=2,alpha=alphas[1], label='- Moderate'),
                   Line2D([0], [0], color=d_colors[2], lw=2,alpha=alphas[2], label='- Mild')]


legend1 = fig.legend(handles=legend_elements1, frameon=False, bbox_to_anchor=(0.1, 0.55),
    loc=3, ncol=4, borderaxespad=0, title= 'Gust Deflections')
legend12 = fig.legend(handles=legend_elements2, frameon=False, bbox_to_anchor=(0.2, 0.5),
    loc=3, ncol=3, borderaxespad=0)

fig.tight_layout()
fig.savefig('plots/other_legend')

In [ ]:
d_colors=['#C0822C','tab:grey','#0F665E']
d_marks = ['^','.','v']
alphas = [1, 0.6, 0.3]
fig, ax = plt.subplots(1,1, figsize=(6,3))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
legend_elements1 = [Line2D([0], [0], color=d_colors[1], lw=2,alpha=0.5, label='Individual test'),
                   Line2D([0], [0], color='k',ls="-", lw=2,alpha=alphas[0], label='Controller average')]


legend1 = fig.legend(handles=legend_elements1, frameon=False, bbox_to_anchor=(0.1, 0.55),
    loc=3, ncol=2, borderaxespad=0)


fig.tight_layout()
fig.savefig('plots/gray_legend')